# Venue data of cities from Foursquare API

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
pd.options.display.max_columns = 500

In [2]:
os.getenv('PATH')

'/Users/robertbozsik/opt/miniconda3/envs/boogie_woogie/bin:/Users/robertbozsik/opt/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin'

In [3]:
os.getenv('PWD')

'/Users/robertbozsik/data-ironhack/project3'

## Foursquare API test with Berlin

In [4]:
client_id = os.getenv('FOURSQUARE_CLIENT_ID')
client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
version = '20180323'

In [5]:
test_limit = 100

In [6]:
# Foursquare API call example - WORKS

url = 'https://api.foursquare.com/v2/venues/explore'

params = (dict(
    client_id=client_id,
    client_secret=client_secret,
    v='20180323',
    #ll='40.7243,-74.0018',
    near='Berlin',
    #query='coffee',
    limit=test_limit
))

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [7]:
#resp.json()

In [8]:
venues = resp.json()

In [9]:
n_results = venues['response']['totalResults']
n_results

220

In [10]:
v_name = venues['response']['groups'][0]['items'][11]['venue']['name']
v_name

'Park am Gleisdreieck - Ostpark'

In [11]:
v_cat_name = venues['response']['groups'][0]['items'][11]['venue']['categories'][0]['name']
v_cat_name

'Park'

In [12]:
len(range(test_limit))

100

In [13]:
v_list = []

for i in range(test_limit):
    v_name = venues['response']['groups'][0]['items'][i]['venue']['name']
    v_cat_name = venues['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
    v_list.append((v_name, v_cat_name))

In [14]:
v_list[0:9]

[('Tiergarten', 'Park'),
 ('Landwehrkanal', 'Canal'),
 ('Urban Nation', 'Art Gallery'),
 ('Tempelhofer Feld', 'Park'),
 ('Café Komine', 'Café'),
 ('Treptower Park', 'Park'),
 ('Classic Remise Berlin', 'Museum'),
 ('Das Stue', 'Hotel'),
 ('Viktoria-Luise-Platz', 'Plaza')]

In [15]:
v_name_list = []
for j in range(test_limit):
    v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
    v_name_list.append(v_name)

In [16]:
v_name_list[0:9]

['Tiergarten',
 'Landwehrkanal',
 'Urban Nation',
 'Tempelhofer Feld',
 'Café Komine',
 'Treptower Park',
 'Classic Remise Berlin',
 'Das Stue',
 'Viktoria-Luise-Platz']

In [17]:
v_cat_name_list = []
for k in range(test_limit):
    v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
    v_cat_name_list.append(v_cat_name)

In [18]:
v_cat_name_list[0:9]

['Park',
 'Canal',
 'Art Gallery',
 'Park',
 'Café',
 'Park',
 'Museum',
 'Hotel',
 'Plaza']

In [19]:
count = [[x, v_cat_name_list.count(x)] for x in set(v_cat_name_list)]

In [20]:
sorted(count)[0:9]

[['Art Gallery', 2],
 ['Art Museum', 1],
 ['Arts & Crafts Store', 1],
 ['Austrian Restaurant', 1],
 ['Bakery', 5],
 ['Bar', 2],
 ['Beer Bar', 2],
 ['Beer Garden', 1],
 ['Beer Store', 1]]

## Create a test DataFrame for Berlin

In [21]:
dict_test = {'city': 'berlin', 
             'venue_name': v_name_list, 
             'venue_category': v_cat_name_list}

In [22]:
#dict_test

In [23]:
pd.DataFrame(dict_test).head()

,city,venue_name,venue_category
0,berlin,Tiergarten,Park
1,berlin,Landwehrkanal,Canal
2,berlin,Urban Nation,Art Gallery
3,berlin,Tempelhofer Feld,Park
4,berlin,Café Komine,Café


## Create a test function for Berlin

In [24]:
def venues_of_a_city(city, lim=100):
    
    '''This function collects the venues of a city from the Foursquare API
    and returns the results as a Pandas DataFrame'''
    
    # Foursquare API
    client_id = os.getenv('FOURSQUARE_CLIENT_ID')
    client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
    version = '20180323'
    
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = (dict(
        client_id=client_id,
        client_secret=client_secret,
        v=version,
        near=city,
        limit=lim
    ))
    
    resp = requests.get(url=url, params=params)
    
    venues = resp.json()
    
    # get the venue names
    v_name_list = []
    for j in range(lim):
        v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
        v_name_list.append(v_name)
    
    # get the venue categories
    v_cat_name_list = []
    for k in range(lim):
        v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
        v_cat_name_list.append(v_cat_name)
    
    # create a dictionary
    dict_berlin = {'city': city, 
                   'venue_name': v_name_list, 
                   'venue_category': v_cat_name_list}
    
    # create a DateFrame
    df_berlin = pd.DataFrame(dict_berlin)
    
    return df_berlin


In [25]:
df_berlin = venues_of_a_city('Berlin')
df_berlin.head()

,city,venue_name,venue_category
0,Berlin,Tiergarten,Park
1,Berlin,Landwehrkanal,Canal
2,Berlin,Urban Nation,Art Gallery
3,Berlin,Tempelhofer Feld,Park
4,Berlin,Café Komine,Café


## Transform the test df

In [26]:
berlin_dummies = pd.get_dummies(df_berlin[['venue_category']])
berlin_dummies['city'] = df_berlin['city']
cols = list(berlin_dummies.columns)
cols = [cols[-1]] + cols[:-1]
berlin_dummies = berlin_dummies[cols]
berlin_dummies.head()

,city,venue_category_Art Gallery,venue_category_Art Museum,venue_category_Arts & Crafts Store,venue_category_Austrian Restaurant,venue_category_Bakery,venue_category_Bar,venue_category_Beer Bar,venue_category_Beer Garden,venue_category_Beer Store,venue_category_Bistro,venue_category_Bookstore,venue_category_Breakfast Spot,venue_category_Café,venue_category_Canal,venue_category_Caucasian Restaurant,venue_category_Climbing Gym,venue_category_Cocktail Bar,venue_category_Coffee Shop,venue_category_Concert Hall,venue_category_Cycle Studio,venue_category_Event Space,venue_category_Falafel Restaurant,venue_category_Farmers Market,venue_category_Food Court,venue_category_Forest,venue_category_French Restaurant,venue_category_Gourmet Shop,venue_category_Hostel,venue_category_Hotel,venue_category_Ice Cream Shop,venue_category_Indie Movie Theater,venue_category_Indie Theater,venue_category_Juice Bar,venue_category_Karaoke Bar,venue_category_Korean Restaurant,venue_category_Mediterranean Restaurant,venue_category_Monument / Landmark,venue_category_Mountain,venue_category_Museum,venue_category_Music Venue,venue_category_Park,venue_category_Pizza Place,venue_category_Plaza,venue_category_Scenic Lookout,venue_category_Spa,venue_category_Spanish Restaurant,venue_category_Vegetarian / Vegan Restaurant,venue_category_Whisky Bar,venue_category_Wine Bar
0,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Berlin,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
berlin_dummies.shape

(100, 50)

In [28]:
berlin_dummies['venue_category_Park'].sum()

13

## Define a function that collects all the venues of the capital cities of Europe

In [29]:
def venue_collector(lst, lim=100):
    
    '''This function collects the venues of a list of cities from the Foursquare API
    and returns the results as a Pandas DataFrame'''
    
    # Foursquare API
    client_id = os.getenv('FOURSQUARE_CLIENT_ID')
    client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
    version = '20180323'
    
    list_of_dfs = []
    
    for city in lst:
        
        url = 'https://api.foursquare.com/v2/venues/explore'

        params = (dict(
            client_id=client_id,
            client_secret=client_secret,
            v=version,
            near=city,
            limit=lim
        ))
    
        resp = requests.get(url=url, params=params)
    
        venues = resp.json()
        
        # number of results of the API response
        n_results = venues['response']['totalResults']
    
        # get the venue names
        v_name_list = []
        for j in range(lim if lim < n_results else n_results): # use the num of results if its less than the limit
            v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
            v_name_list.append(v_name)
    
        # get the venue categories
        v_cat_name_list = []
        for k in range(lim if lim < n_results else n_results):
            v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
            v_cat_name_list.append(v_cat_name)
    
        # create a dictionary
        dict_city = {'city': city, 
                       'venue_name': v_name_list, 
                       'venue_category': v_cat_name_list}
    
        # create a DateFrame
        df_city = pd.DataFrame(dict_city)
        
        # append it the the list of DataFrames
        list_of_dfs.append(df_city)
    
    # concatenate all DataFrames
    df = pd.concat(list_of_dfs).reset_index(drop=True)
    
    return df

## Test the function

In [30]:
test_cap_cities_europe = ['Amsterdam',
                          'Athens',
                          'Andorra la Vella',
                          'Belgrade',
                          'Berlin',
                          'Bern']

In [31]:
test_cap_venues = venue_collector(test_cap_cities_europe)

test_cap_venues.head()

,city,venue_name,venue_category
0,Amsterdam,Vondelpark,Park
1,Amsterdam,Amsterdamse Grachten,Canal
2,Amsterdam,Concerto Records,Record Shop
3,Amsterdam,Zoku,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place


## Run the function on all the capital cities of Europe

In [32]:
cap_cities_europe = ['Amsterdam',
                     'Andorra la Vella',
                     'Athens',
                     'Belgrade',
                     'Berlin',
                     'Bern',
                     'Bratislava',
                     'Brussels',
                     'Bucharest',
                     'Budapest',
                     'Chisinau',
                     'Copenhagen',
                     'Dublin',
                     'Helsinki',
                     'Kiev',
                     'Lisbon',
                     'Ljubljana',
                     'London',
                     'Luxembourg',
                     'Madrid',
                     'Minsk',
                     'Monaco',
                     'Moscow',
                     'Nicosia',
                     'Nuuk',
                     'Oslo',
                     'Paris',
                     'Podgorica',
                     'Prague',
                     'Reykjavik',
                     'Riga',
                     'Rome',
                     'San Marino',
                     'Sarajevo',
                     'Skopje',
                     'Sofia',
                     'Stockholm',
                     'Tallinn',
                     'Tirana',
                     'Vaduz',
                     'Valletta',
                     'Vatican City',
                     'Vienna',
                     'Vilnius',
                     'Warsaw',
                     'Zagreb'] # 'Zurich'

In [33]:
len(cap_cities_europe)

46

In [34]:
cap_cities_europe_venues = venue_collector(cap_cities_europe)

In [35]:
cap_cities_europe_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4307 entries, 0 to 4306
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   city            4307 non-null   object
 1   venue_name      4307 non-null   object
 2   venue_category  4307 non-null   object
dtypes: object(3)
memory usage: 101.1+ KB


In [36]:
cap_cities_europe_venues.head()

,city,venue_name,venue_category
0,Amsterdam,Vondelpark,Park
1,Amsterdam,Amsterdamse Grachten,Canal
2,Amsterdam,Concerto Records,Record Shop
3,Amsterdam,Zoku,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place


## Group by cities and venue categories

In [37]:
df = cap_cities_europe_venues.copy()

In [38]:
df['venue_count'] = 1

In [39]:
df.head()

,city,venue_name,venue_category,venue_count
0,Amsterdam,Vondelpark,Park,1
1,Amsterdam,Amsterdamse Grachten,Canal,1
2,Amsterdam,Concerto Records,Record Shop,1
3,Amsterdam,Zoku,Hotel,1
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place,1


In [40]:
df.isna().sum()

city              0
venue_name        0
venue_category    0
venue_count       0
dtype: int64

In [41]:
df = df.groupby(['city', 'venue_category']).agg({'venue_count': 'sum'})

In [42]:
df = df.reset_index()

In [43]:
df.head()

,city,venue_category,venue_count
0,Amsterdam,Arcade,1
1,Amsterdam,Art Museum,1
2,Amsterdam,BBQ Joint,1
3,Amsterdam,Bakery,2
4,Amsterdam,Bar,2


**Check if the number of parks in Berlin is still 13**

In [44]:
df[(df['city'] == 'Berlin') & (df['venue_category'] == 'Park')]

,city,venue_category,venue_count
243,Berlin,Park,13


## Tranform the venue categories to columns

In [45]:
df2 = df.copy()

In [46]:
df2 = df2.pivot(index='city', columns='venue_category', values='venue_count')

In [47]:
df2.shape

(46, 351)

In [48]:
df2.head()

venue_category,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paella Restaurant,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souv

In [49]:
df2 = df2.reset_index()

In [50]:
df2.head()

venue_category,city,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paella Restaurant,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant

In [51]:
df2.tail()

venue_category,city,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paella Restaurant,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant

In [52]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Columns: 352 entries, city to Zoo Exhibit
dtypes: float64(351), object(1)
memory usage: 126.6+ KB


## Export to JSON format

In [53]:
df2.to_json('../data/Foursquare_venues_data.json')

In [54]:
check = pd.read_json('../data/Foursquare_venues_data.json')
check.head()

,city,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paella Restaurant,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop